The output is a batch of feature vectors. For each input image, the feature vector has size num_features = 2048. The feature vectors can then be used further, e.g., for classification as above.

The input images are expected to have color values in the range [0,1], following the common image input conventions. The expected size of the input images is height x width = 299 x 299 pixels by default, but other input sizes are possible (within limits).



In [19]:
%reset -f

In [1]:
dataset_path = "C:/Users/vaibhav.jadhav/Downloads/seed_4_dataset/seed_4"

In [2]:
spectrogram_height = 229
spectrogram_width = 229
hop_size = 3

frame_size = ((spectrogram_height-1) * 2)
sample_size = ((spectrogram_width-1) * hop_size)

In [3]:
fs = 200

In [4]:
import os
import sys
import math
import numpy as np
# import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.io import loadmat

In [5]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import tensorflow_hub as hub

In [7]:
import gc

In [8]:
import time

In [9]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [10]:
def compute_spectrogram(signal, sr=fs,frame_size = frame_size, hop = hop_size):
    stft = librosa.stft(signal, n_fft=frame_size, hop_length=hop)
    log_stft= librosa.power_to_db(np.abs(stft) ** 2)
    return log_stft
#     print(log_stft.shape)
#     plot_spectrogram(log_stft, sr, hop, y_axis="log")

In [11]:
def plot_spectrogram(Y, sr=fs, hop_length=hop_size, y_axis="linear"):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y, 
                             sr=sr, 
                             hop_length=hop_length, 
                             x_axis="time", 
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")

In [12]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]

In [13]:
session_label_dict = {}
session_label_dict[1] = session1_label
session_label_dict[2] = session2_label
session_label_dict[3] = session3_label
session_label_dict

{1: [1, 2, 3, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 3, 2, 2, 3, 3, 0, 3, 0, 3],
 2: [2, 1, 3, 0, 0, 2, 0, 2, 3, 3, 2, 3, 2, 0, 1, 1, 2, 1, 0, 3, 0, 1, 3, 1],
 3: [1, 2, 2, 1, 3, 3, 3, 1, 1, 2, 1, 0, 2, 3, 3, 0, 2, 3, 0, 0, 2, 0, 1, 0]}

In [14]:
all_trial_sizes = {}
for exp in range(1,4):
    all_trial_sizes[exp] = []

In [15]:
fs = 200

def decompose_ram_efficient(file, file_mat, exp_id):
    t_start = time.time()
    # trial*channel*sample
    feature_extractor_model = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
    out_dim_of_TL_model = 2048
    data = loadmat(file)
    frequency = fs
#     time_window = 0.5
#     data_points_per_sample = frequency * time_window  #fz = 200per sec -> 100 data_points per 0.5sec window
    data_points_per_sample = sample_size
    name = ""
    for char in list(data.keys())[7]: #instead of 7, u can pass any no. between 3 and 15
      if char == "_":
        break
      name = name+char

#     fft_crop_length = 21 

    decomposed_specgram = np.empty([0, 62, 229, 229, 3])
    label = np.array([])


    x_TL_features = np.empty([0, 62, out_dim_of_TL_model])
    
    
    for trial in range(24):
        tmp_trial_signal = data[name + '_eeg' + str(trial + 1)]
        num_sample = int(len(tmp_trial_signal[0]) / data_points_per_sample) 
        print('{}-{}'.format(trial + 1, num_sample))
        all_trial_sizes[exp_id].append(num_sample)

        # temp_fft = np.empty([0, num_sample, 21])
        temp_specgram = np.empty([0, 229, 229])
        label = np.append(label, [session_label_dict[exp_id][trial]] * num_sample)

        t1_spec  =time.time()
        for channel in range(62):
            trial_signal = tmp_trial_signal[channel]

            delta_theta_alpha = butter_bandpass_filter(trial_signal, 1, 14, frequency, order=3)
            delta_theta_alpha = delta_theta_alpha+abs(np.min(delta_theta_alpha))
            delta_theta_alpha = delta_theta_alpha/np.max(delta_theta_alpha)
            
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            beta = beta+abs(np.min(beta))
            beta = beta/np.max(beta)
            
            gamma = butter_bandpass_filter(trial_signal, 31, 51, frequency, order=3)
            gamma = gamma+abs(np.min(gamma))
            gamma = gamma/np.max(gamma)

            specgram_delta_theta_alpha = np.zeros(shape=[0, 229, 229], dtype=float)
            specgram_beta = np.zeros(shape=[0, 229, 229], dtype=float)
            specgram_gamma = np.zeros(shape=[0, 229, 229], dtype=float)

            for index in range(num_sample):
                specgram_delta_theta_alpha = np.vstack([specgram_delta_theta_alpha, compute_spectrogram(delta_theta_alpha[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                specgram_beta = np.vstack([specgram_beta, compute_spectrogram(beta[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                specgram_gamma = np.vstack([specgram_gamma, compute_spectrogram(gamma[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                # print(fft_delta.shape, fft_theta.shape, fft_alpha.shape, fft_beta.shape, fft_gamma.shape)

            temp_specgram = np.vstack([temp_specgram, specgram_delta_theta_alpha])
            temp_specgram = np.vstack([temp_specgram, specgram_beta])
            temp_specgram = np.vstack([temp_specgram, specgram_gamma])

        t2_spec = time.time()
        temp_trial_specgram = temp_specgram.reshape(-1, 3, num_sample, 229, 229)
        temp_trial_specgram = temp_trial_specgram.transpose([2, 0, 3, 4, 1])
        subject_id = ""
        for i in file_mat:
            if i == "_":
                break
            subject_id += i
#         np.save(f"{dataset_path}/my_extracted_features/spectrogram(229_229_3)/{subject_id}_{trial+1}_X_1D.npy",temp_trial_specgram)
#         decomposed_specgram = np.vstack([decomposed_specgram, temp_trial_specgram])
#         

        subject = subject_id
#         trial_group = en+1


        temp_x = temp_trial_specgram #(#,62,229,229,3)
        print(f"time for subject:{subject}, exp:{exp_id} and trial:{trial+1} for specgrams is: {t2_spec - t1_spec}")
        print(f"subject:{subject}, exp:{exp_id} and trial:{trial+1} has specgrams as: {temp_x.shape}")
        
        # spectrogram creation complete
        
############################# TL start ##################################    
        # starting Transefer learning, forward pass in inception v3 with top layer removed
        t1_TL = time.time()
        pretrained_model_without_top_layer = hub.KerasLayer(
            feature_extractor_model, input_shape=(229, 229, 3), trainable=False)

        
        for sample in range(temp_x.shape[0]):
            temp_x_TL_features = np.empty([0, out_dim_of_TL_model])
            for channel in range(temp_x.shape[1]):
                results = ((np.array(pretrained_model_without_top_layer(temp_x[sample,channel].reshape((1,229,229,3))))))
                temp_x_TL_features = np.vstack([temp_x_TL_features, results])
            temp_x_TL_features = temp_x_TL_features.reshape((1,temp_x_TL_features.shape[0],out_dim_of_TL_model))
            x_TL_features = np.vstack([x_TL_features, temp_x_TL_features])
        
        t2_TL = time.time()
        print(f"time for subject:{subject}, exp:{exp_id} and trial:{trial+1} for TL is: {t2_TL - t1_TL}")
        temp_x = None 
        temp_trial_specgram = None
        gc.collect()
############################# TL done ##################################        
    
####converting EEG signals to 8x9 feature map, to capture their spatial information and pass to CNN model later ####
####for more information about 8x9 map, refer to paper with following link or DOI ####
# link: https://www.itm-conferences.org/articles/itmconf/abs/2023/03/itmconf_icdsia2023_02011/itmconf_icdsia2023_02011.html
# DOI: https://doi.org/10.1051/itmconf/20235302011        
    print(f"TL_features_for subject:{subject} and exp:{exp_id} -> {x_TL_features.shape}")

    t1_x89 = time.time()
    X89 = np.zeros((x_TL_features.shape[0], 8, 9, out_dim_of_TL_model))
    
    X89[:, 0, 2, :] = x_TL_features[:, 3, :]
    X89[:, 0, 3:6, :] = x_TL_features[:, 0:3, :]
    X89[:, 0, 6, :] = x_TL_features[:, 4, :]

    for i in range(5):
      X89[:, i + 1, :, :] = x_TL_features[:, 5 + i * 9:5 + (i + 1) * 9, :]

    X89[:, 6, 1:8, :] = x_TL_features[:, 50:57, :]
    X89[:, 7, 2:7, :] = x_TL_features[:, 57:62, :]
    
    t2_x89 = time.time()
    print(f"time for 8*9 map for subject:{subject} and exp:{exp_id} is -> {t2_x89 - t1_x89}")
    print(f"8*9 map dim for subject:{subject} and exp:{exp_id} is -> {X89.shape}")


    np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy",X89)
    np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_y.npy",label)

    t_end = time.time()
    print(f"time for subject:{subject} and exp:{exp_id} is -> {t_end - t_start} sec.")
#     x89 = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_mobilenet_v2/{subject}_{trial_group}_X89.npy")
#     print(x89.shape)
    print("######################################################")


#         %reset -f
    

In [16]:
eeg_raw_file_path = f"{dataset_path}/eeg_raw_data/"


In [17]:
session_file_dict = {}
for exp in range(1,4):
    files = os.listdir(f"{eeg_raw_file_path}{str(exp)}")
    files.sort(key= (lambda i : i[:2] if i[1]!="_" else "0"+i[0]) )
    session_file_dict[exp] = files

In [18]:
session_file_dict

{1: ['1_20160518.mat',
  '2_20150915.mat',
  '3_20150919.mat',
  '4_20151111.mat',
  '5_20160406.mat',
  '6_20150507.mat',
  '7_20150715.mat',
  '8_20151103.mat',
  '9_20151028.mat',
  '10_20151014.mat',
  '11_20150916.mat',
  '12_20150725.mat',
  '13_20151115.mat',
  '14_20151205.mat',
  '15_20150508.mat'],
 2: ['1_20161125.mat',
  '2_20150920.mat',
  '3_20151018.mat',
  '4_20151118.mat',
  '5_20160413.mat',
  '6_20150511.mat',
  '7_20150717.mat',
  '8_20151110.mat',
  '9_20151119.mat',
  '10_20151021.mat',
  '11_20150921.mat',
  '12_20150804.mat',
  '13_20151125.mat',
  '14_20151208.mat',
  '15_20150514.mat'],
 3: ['1_20161126.mat',
  '2_20151012.mat',
  '3_20151101.mat',
  '4_20151123.mat',
  '5_20160420.mat',
  '6_20150512.mat',
  '7_20150721.mat',
  '8_20151117.mat',
  '9_20151209.mat',
  '10_20151023.mat',
  '11_20151011.mat',
  '12_20150807.mat',
  '13_20161130.mat',
  '14_20151215.mat',
  '15_20150527.mat']}

for subject in range(13,14):
    for exp in range(2,4):
        file_name = f"{eeg_raw_file_path}{str(exp)}/{session_file_dict[exp][subject]}"
        print(f'processing {session_file_dict[exp][subject]}')
        decompose_ram_efficient(file_name, session_file_dict[exp][subject], exp)
    for exp in range(1,4):
        all_trial_sizes[exp] = []
    print("###########################################################")

In [20]:
for subject in range(0,15):
    for exp in range(1,4):
        file_name = f"{eeg_raw_file_path}{str(exp)}/{session_file_dict[exp][subject]}"
        print(f'processing {session_file_dict[exp][subject]}')
        decompose_ram_efficient(file_name, session_file_dict[exp][subject], exp)
    for exp in range(1,4):
        all_trial_sizes[exp] = []
    print("###########################################################")

processing 15_20150508.mat
1-49
time for subject:15, exp:1 and trial:1 for specgrams is: 142.44820833206177
subject:15, exp:1 and trial:1 has specgrams as: (49, 62, 229, 229, 3)
time for subject:15, exp:1 and trial:1 for TL is: 234.29948210716248
2-27


KeyboardInterrupt: 

In [ ]:
trail_samples = {1:68,2:68,3:60,4:69,5:54,6:57,7:69,8:63,9:77,10:69,11:68,12:68,13:68,14:69,15:60}

In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
